In [1]:
%%capture
!pip install pandas bertopic

In [6]:
%%capture
!conda install -n base pandas bertopic 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import pandas as pd
from bertopic import BERTopic

# Load the comments dataset
df = pd.read_csv("../comments_dataset.csv") 

# Assuming your CSV has a column named 'text' containing the comments
comments = df['text'].tolist()

# Create and fit the BERTopic model
model = BERTopic()
topics, probabilities = model.fit_transform(comments)

# Print the top terms per topic
for topic in model.get_topic_info():
    print(topic)

print(model.get_topic_info())
# Visualize the topics (optional)
# model.visualize_topics()

['Watch this episode fully uncensored &amp; ad-free on Patreon: <a href="https://patreon.com/dannyjones">https://patreon.com/dannyjones</a><br>Support the show by checking out our sponsors:<br><a href="https://bit.ly/viiadannyjones">https://bit.ly/viiadannyjones</a> - Try VIIA &amp; use code DANNY<br><a href="https://whiterabbitenergy.com/?ref=DJP">https://whiterabbitenergy.com/?ref=DJP</a> - Use code DJP for 20% off', 'If they took vaccines off the market 1/2 the country would lose their shit.  These two lunatics want to sacrifice the good for the perfect.', 'Calley how many more babies and parents have to die while youre busy with niceties? Thats the issue', 'Why would Jack turn down Rogan and Tucker? He could call them out just like he did this guy', 'TRUST NOT... anyone who <br>Slowly Measures their speech... Correct Jack... That person is FOS,  just as Jack said in 1st 4 minutes!  A young un-wise guy vs 35+ years of Wisdom.... on display!  jmo', 'Transparency bro', 'OMG  Danny tha